In [13]:
import json
import random
import requests
import time
from datetime import datetime, timedelta

In [14]:
ORION_URL = "http://localhost:1026"

headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/ld+json',
        'Link': '<https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"'
    }

response = requests.get(f"{ORION_URL}/version")
print(response.json())

{'orionld version': '1.4.0', 'orion version': '1.15.0-next', 'uptime': '0 d, 1 h, 5 m, 0 s', 'git_hash': 'nogitversion', 'compile_time': 'Sat Aug 26 06:19:09 UTC 2023', 'compiled_by': 'root', 'compiled_in': '', 'release_date': 'Sat Aug 26 06:19:09 UTC 2023', 'doc': 'https://fiware-orion.readthedocs.org/en/master/'}


In [15]:
def generate_single_charging_session_entity(session_number):
    """
    Generate a single charging session test entity
    """
    base_time = datetime(2025, 1, 28, 8, 0, 0)
    session_statuses = ["ended", "active", "suspended", "preparing"]
    charging_point_prefixes = [
        "ParisSaclay30-1-P-",
        "ParisSaclay30-2-P-", 
        "ParisSaclay30-3-P-"
    ]
    
    # Generate a unique ID with a timestamp to avoid duplication 生成包含时间戳的唯一ID，避免重复
    timestamp = int(time.time() * 1000)  # Millisecond timestamp 毫秒时间戳
    session_id = f"{session_number:03d}_{timestamp}"
    charging_point_prefix = random.choice(charging_point_prefixes)
    charging_point_id = f"{charging_point_prefix}{random.randint(1, 20):02d}"
    status = random.choice(session_statuses)
    
    # Generate random times
    start_time_offset = timedelta(
        days=random.randint(0, 30),
        hours=random.randint(0, 23),
        minutes=random.randint(0, 59)
    )
    start_time = base_time + start_time_offset
    
    session_duration = timedelta(
        hours=random.randint(0, 8),
        minutes=random.randint(30, 59)
    )
    end_time = start_time + session_duration
    
    entity = {
        "id": f"urn:ngsi-ld:ChargingSessionTest:session-{session_id}",
        "type": "ChargingSessionTest",
        
        "refChargingPoint": {
            "type": "Relationship",
            "object": f"urn:ngsi-ld:ChargingPoint:{charging_point_id}"
        },
        
        "sessionId": {
            "type": "Property",
            "value": session_id
        },
        
        "sessionStatus": {
            "type": "Property",
            "value": status
        },
        
        "sessionStartTime": {
            "type": "Property",
            "value": {
                "@type": "DateTime",
                "@value": start_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
            },
            "observedAt": start_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
        },
        
        "sessionEndTime": {
            "type": "Property",
            "value": {
                "@type": "DateTime",
                "@value": end_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
            },
            "observedAt": end_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
        }
    }
    
    return entity

def create_single_entity_with_timing(entity, orion_url="http://localhost:1026"):
    """
    Create a single entity in Orion and measure the time
    """
    headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/ld+json',
            'Link': '<https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"'
    }
    
    start_time = time.time()
    
    try:
        response = requests.post(
            f"{orion_url}/ngsi-ld/v1/entities",
            headers=headers,
            data=json.dumps(entity),
            timeout=30
        )
        
        end_time = time.time()
        elapsed_ms = (end_time - start_time) * 1000
        
        success = response.status_code == 201
        
        # 处理409错误（实体已存在）
        if response.status_code == 409:
            return False, elapsed_ms, response.status_code, "Entity already exists"
        
        return success, elapsed_ms, response.status_code, response.text
        
    except Exception as e:
        end_time = time.time()
        elapsed_ms = (end_time - start_time) * 1000
        return False, elapsed_ms, "ERROR", str(e)

def delete_single_entity_with_timing(entity_id, orion_url="http://localhost:1026"):
    """
    删除单个实体并测量时间 (类似于 create_single_entity_with_timing)
    Delete a single entity and measure timing (similar to create_single_entity_with_timing)
    """
    start_time = time.time()
    
    try:
        response = requests.delete(
            f"{orion_url}/ngsi-ld/v1/entities/{entity_id}",
            timeout=30
        )
        
        end_time = time.time()
        elapsed_ms = (end_time - start_time) * 1000
        
        # 删除成功的状态码是204，不是201
        # Success status code for delete is 204, not 201
        success = response.status_code == 204
        
        # 处理404错误（实体不存在）
        # Handle 404 error (entity not found)
        if response.status_code == 404:
            return False, elapsed_ms, response.status_code, "Entity not found"
        
        return success, elapsed_ms, response.status_code, response.text
        
    except Exception as e:
        end_time = time.time()
        elapsed_ms = (end_time - start_time) * 1000
        return False, elapsed_ms, "ERROR", str(e)

def get_test_entities(orion_url="http://localhost:1026"):

    """
    获取所有测试实体用于删除 (修改自 clear_test_entities)
    Get all test entities for deletion (modified from clear_test_entities)
    """
    try:
        # 查询所有ChargingSessionTest类型的实体
        # Query all ChargingSessionTest type entities
        response = requests.get(
            f"{orion_url}/ngsi-ld/v1/entities",
            params={"type": "ChargingSessionTest", "limit": 1000},
            headers={"Accept": "application/json"}
        )
        
        if response.status_code == 200:
            entities = response.json()
            entity_ids = [entity["id"] for entity in entities]
            
            print(f"Found {len(entity_ids)} entities for deletion")
            
            # 显示前几个实体ID作为示例 (Show first few entity IDs as examples)
            if entity_ids:
                print("Sample entity IDs:")
                for i, entity_id in enumerate(entity_ids[:3]):
                    print(f"  {i+1}. {entity_id}")
                if len(entity_ids) > 3:
                    print(f"  ... and {len(entity_ids)-3} more")
            
            return entity_ids
        else:
            print(f"Failed to query entities: {response.status_code}")
            return []
            
    except Exception as e:
        print(f"Error querying entities: {e}")
        return []

def run_create_performance_test(iterations=20, orion_url="http://localhost:1026"):
    """
    运行创建性能测试
    Run create performance test 
    """
    results = []
    
    print(f"Starting create performance test: {iterations} iterations")
    print("=" * 60)
    
    for i in range(1, iterations + 1):
        print(f"Create {i:2d}/{iterations}: ", end="")
        
        # 生成实体 (Generate entity)
        entity = generate_single_charging_session_entity(i)
        entity_id = entity["sessionId"]["value"]  # 统一使用 entity_id
        
        # 创建实体并计时 (Create entity with timing)
        success, elapsed_ms, response_code, error_message = create_single_entity_with_timing(entity, orion_url)
        
        # 记录结果 (Record result)
        result = {
            "iteration": i,
            "entity_id": entity_id,           # 统一命名：entity_id 而不是 session_id
            "success": success,
            "elapsed_ms": elapsed_ms,
            "response_code": response_code,
            "error_message": error_message,
            "timestamp": datetime.now().isoformat()
        }
        results.append(result)
        
        # 打印即时结果 (Print immediate result)
        status_icon = "ok" if success else "fail"
        print(f"{status_icon} {entity_id} - {elapsed_ms:.1f}ms - {response_code}")
        
        # 请求间隔 (Request interval)
        time.sleep(0.1)
    
    return results

def run_delete_performance_test(entity_ids, orion_url="http://localhost:1026"):
    """
    运行删除性能测试 
    Run delete performance test
    """
    results = []
    iterations = len(entity_ids)  # 统一使用 iterations 表示总数
    
    print(f"Starting delete performance test: {iterations} iterations")
    print("=" * 60)
    
    for i, full_entity_id in enumerate(entity_ids, 1):
        print(f"Delete {i:2d}/{iterations}: ", end="")
        
        # 提取简短的entity_id用于显示 (Extract short entity_id for display)
        entity_id = full_entity_id.split(":")[-1]  # 统一命名格式
        
        # 删除实体并计时 (Delete entity with timing)
        success, elapsed_ms, response_code, error_message = delete_single_entity_with_timing(full_entity_id, orion_url)
        
        # 记录结果 (Record result)
        result = {
            "iteration": i,                   
            "entity_id": entity_id,           
            "success": success,               
            "elapsed_ms": elapsed_ms,         
            "response_code": response_code,   
            "error_message": error_message,   
            "timestamp": datetime.now().isoformat()  
        }
        results.append(result)
        
        # 打印即时结果 (Print immediate result)
        status_icon = "ok" if success else "fail"
        print(f"{status_icon} {entity_id} - {elapsed_ms:.1f}ms - {response_code}")
        
        # 请求间隔 (Request interval)
        time.sleep(0.1)
    
    return results

def analyze_performance_results(results, operation_type="OPERATION"):
    """
    分析性能结果 (统一分析函数，支持创建和删除)
    Analyze performance results (unified analysis function for both create and delete)
    """
    if not results:
        print("No results to analyze")
        return
    
    successful_results = [r for r in results if r["success"]]
    failed_results = [r for r in results if not r["success"]]
    
    print("\n" + "=" * 60)
    print(f"{operation_type} PERFORMANCE ANALYSIS")
    print("=" * 60)
    
    total_iterations = len(results)
    success_count = len(successful_results)
    failure_count = len(failed_results)
    success_rate = (success_count / total_iterations) * 100 if total_iterations > 0 else 0
    
    print(f"Total iterations:      {total_iterations}")
    print(f"Successful:            {success_count} ({success_rate:.1f}%)")
    print(f"Failed:                {failure_count} ({100-success_rate:.1f}%)")
    
    if successful_results:
        response_times = [r["elapsed_ms"] for r in successful_results]
        
        min_time = min(response_times)
        max_time = max(response_times)
        avg_time = sum(response_times) / len(response_times)
        
        sorted_times = sorted(response_times)
        p50 = sorted_times[len(sorted_times) // 2]
        p95_index = int(len(sorted_times) * 0.95)
        p95 = sorted_times[min(p95_index, len(sorted_times) - 1)]
        
        print(f"\nResponse Time Statistics (ms):")
        print(f"Minimum:               {min_time:.1f} ms")
        print(f"Maximum:               {max_time:.1f} ms")
        print(f"Average:               {avg_time:.1f} ms")
        print(f"Median (P50):          {p50:.1f} ms")
        print(f"95th Percentile:       {p95:.1f} ms")
        

    
    if failed_results:
        print(f"\n First 3 Failed Operations:")
        for result in failed_results[:3]:
            print(f"   {result['iteration']:2d}. {result['entity_id']} - {result['response_code']}")
            if result['error_message']:
                print(f"       Error: {result['error_message'][:100]}...")

def generate_update_data():
    """
    生成用于更新实体的随机数据
    Generate random data for updating entities
    """
    session_statuses = ["ended", "active", "suspended", "preparing"]
    
    update_data = {
        "sessionStatus": {
            "type": "Property",
            "value": random.choice(session_statuses)
        }
    }
    
    return update_data

def update_single_entity_with_timing(full_entity_id, orion_url="http://localhost:1026"):
    """
    更新单个实体并测量时间
    Update a single entity and measure timing
    """
    update_data = generate_update_data()
        
    start_time = time.time()
    
    try:
        response = requests.patch(
            f"{orion_url}/ngsi-ld/v1/entities/{full_entity_id}/attrs",
            headers=headers,
            data=json.dumps(update_data),
            timeout=30
        )
        
        end_time = time.time()
        elapsed_ms = (end_time - start_time) * 1000
        
        success = response.status_code == 204
        
        if response.status_code == 404:
            return False, elapsed_ms, response.status_code, "Entity not found"
        
        return success, elapsed_ms, response.status_code, response.text
        
    except Exception as e:
        end_time = time.time()
        elapsed_ms = (end_time - start_time) * 1000
        return False, elapsed_ms, "ERROR", str(e)

def run_update_performance_test(orion_url="http://localhost:1026"):
    """
    运行更新性能测试
    Run update performance test
    """
    # 获取现有的测试实体
    try:
        response = requests.get(
            f"{orion_url}/ngsi-ld/v1/entities",
            params={"type": "ChargingSessionTest", "limit": 1000},
            headers={"Accept": "application/json"}
        )
        
        if response.status_code == 200:
            entities = response.json()
            entity_ids = [entity["id"] for entity in entities]
            
            if not entity_ids:
                print("No test entities found to update")
                return []
                
        else:
            print(f"Failed to query entities: {response.status_code}")
            return []
            
    except Exception as e:
        print(f"Error querying entities: {e}")
        return []
    
    # 运行更新测试
    results = []
    iterations = len(entity_ids)
    
    print(f"Starting update performance test: {iterations} iterations")
    print("=" * 60)
    
    for i, full_entity_id in enumerate(entity_ids, 1):
        print(f"Update {i:2d}/{iterations}: ", end="")
        
        entity_id = full_entity_id.split(":")[-1]
        
        success, elapsed_ms, response_code, error_message = update_single_entity_with_timing(full_entity_id, orion_url)
        
        result = {
            "iteration": i,
            "entity_id": entity_id,
            "success": success,
            "elapsed_ms": elapsed_ms,
            "response_code": response_code,
            "error_message": error_message,
            "timestamp": datetime.now().isoformat()
        }
        results.append(result)
        
        status_icon = "ok" if success else "fail"
        print(f"{status_icon} {entity_id} - {elapsed_ms:.1f}ms - {response_code}")
        
        time.sleep(0.1)
    
    return results


In [16]:
entity_ids = get_test_entities()

Found 20 entities for deletion
Sample entity IDs:
  1. urn:ngsi-ld:ChargingSessionTest:session-001_1751881102159
  2. urn:ngsi-ld:ChargingSessionTest:session-002_1751881102301
  3. urn:ngsi-ld:ChargingSessionTest:session-003_1751881102413
  ... and 17 more


In [17]:
delete_results = run_delete_performance_test(entity_ids)

Starting delete performance test: 20 iterations
Delete  1/20: ok session-001_1751881102159 - 26.9ms - 204
Delete  2/20: ok session-002_1751881102301 - 9.3ms - 204
Delete  3/20: ok session-003_1751881102413 - 17.9ms - 204
Delete  4/20: ok session-004_1751881102524 - 18.3ms - 204
Delete  5/20: ok session-005_1751881102644 - 17.3ms - 204
Delete  6/20: ok session-006_1751881102763 - 15.4ms - 204
Delete  7/20: ok session-007_1751881102884 - 18.2ms - 204
Delete  8/20: ok session-008_1751881103005 - 12.7ms - 204
Delete  9/20: ok session-009_1751881103126 - 18.1ms - 204
Delete 10/20: ok session-010_1751881103254 - 16.5ms - 204
Delete 11/20: ok session-011_1751881103376 - 18.2ms - 204
Delete 12/20: ok session-012_1751881103494 - 18.3ms - 204
Delete 13/20: ok session-013_1751881103614 - 17.3ms - 204
Delete 14/20: ok session-014_1751881103733 - 16.2ms - 204
Delete 15/20: ok session-015_1751881103857 - 17.2ms - 204
Delete 16/20: ok session-016_1751881103982 - 17.2ms - 204
Delete 17/20: ok session-

In [18]:
print("delete performance:")  
analyze_performance_results(delete_results)  

delete performance:

OPERATION PERFORMANCE ANALYSIS
Total iterations:      20
Successful:            20 (100.0%)
Failed:                0 (0.0%)

Response Time Statistics (ms):
Minimum:               9.3 ms
Maximum:               26.9 ms
Average:               16.8 ms
Median (P50):          17.3 ms
95th Percentile:       26.9 ms


In [19]:
test_entity = generate_single_charging_session_entity(1)
print('Generated entity:')
print(json.dumps(test_entity, indent=2))

Generated entity:
{
  "id": "urn:ngsi-ld:ChargingSessionTest:session-001_1751881256825",
  "type": "ChargingSessionTest",
  "refChargingPoint": {
    "type": "Relationship",
    "object": "urn:ngsi-ld:ChargingPoint:ParisSaclay30-1-P-15"
  },
  "sessionId": {
    "type": "Property",
    "value": "001_1751881256825"
  },
  "sessionStatus": {
    "type": "Property",
    "value": "suspended"
  },
  "sessionStartTime": {
    "type": "Property",
    "value": {
      "@type": "DateTime",
      "@value": "2025-02-01T06:39:00.000Z"
    },
    "observedAt": "2025-02-01T06:39:00.000Z"
  },
  "sessionEndTime": {
    "type": "Property",
    "value": {
      "@type": "DateTime",
      "@value": "2025-02-01T07:22:00.000Z"
    },
    "observedAt": "2025-02-01T07:22:00.000Z"
  }
}


In [20]:
create_results = run_create_performance_test(20)  

Starting create performance test: 20 iterations
Create  1/20: ok 001_1751881256847 - 25.2ms - 201


Create  2/20: ok 002_1751881256972 - 12.8ms - 201
Create  3/20: ok 003_1751881257086 - 20.9ms - 201
Create  4/20: ok 004_1751881257208 - 20.4ms - 201
Create  5/20: ok 005_1751881257329 - 20.2ms - 201
Create  6/20: ok 006_1751881257450 - 11.1ms - 201
Create  7/20: ok 007_1751881257562 - 19.9ms - 201
Create  8/20: ok 008_1751881257682 - 11.6ms - 201
Create  9/20: ok 009_1751881257794 - 19.6ms - 201
Create 10/20: ok 010_1751881257915 - 12.6ms - 201
Create 11/20: ok 011_1751881258028 - 10.9ms - 201
Create 12/20: ok 012_1751881258140 - 19.9ms - 201
Create 13/20: ok 013_1751881258260 - 10.2ms - 201
Create 14/20: ok 014_1751881258371 - 17.8ms - 201
Create 15/20: ok 015_1751881258489 - 19.7ms - 201
Create 16/20: ok 016_1751881258610 - 14.1ms - 201
Create 17/20: ok 017_1751881258724 - 21.3ms - 201
Create 18/20: ok 018_1751881258846 - 10.6ms - 201
Create 19/20: ok 019_1751881258957 - 9.0ms - 201
Create 20/20: ok 020_1751881259067 - 10.8ms - 201


In [21]:
print("\ncreate proformance:")
analyze_performance_results(create_results)


create proformance:

OPERATION PERFORMANCE ANALYSIS
Total iterations:      20
Successful:            20 (100.0%)
Failed:                0 (0.0%)

Response Time Statistics (ms):
Minimum:               9.0 ms
Maximum:               25.2 ms
Average:               15.9 ms
Median (P50):          17.8 ms
95th Percentile:       25.2 ms


In [22]:
entity_ids = get_test_entities()

Found 20 entities for deletion
Sample entity IDs:
  1. urn:ngsi-ld:ChargingSessionTest:session-001_1751881256847
  2. urn:ngsi-ld:ChargingSessionTest:session-002_1751881256972
  3. urn:ngsi-ld:ChargingSessionTest:session-003_1751881257086
  ... and 17 more


In [23]:
update_results = run_update_performance_test()

Starting update performance test: 20 iterations
Update  1/20: ok session-001_1751881256847 - 13.4ms - 204
Update  2/20: ok session-002_1751881256972 - 18.7ms - 204
Update  3/20: ok session-003_1751881257086 - 15.6ms - 204
Update  4/20: ok session-004_1751881257208 - 11.9ms - 204
Update  5/20: ok session-005_1751881257329 - 23.7ms - 204
Update  6/20: ok session-006_1751881257450 - 24.9ms - 204
Update  7/20: ok session-007_1751881257562 - 12.5ms - 204
Update  8/20: ok session-008_1751881257682 - 19.5ms - 204
Update  9/20: ok session-009_1751881257794 - 19.9ms - 204
Update 10/20: ok session-010_1751881257915 - 12.4ms - 204
Update 11/20: ok session-011_1751881258028 - 14.1ms - 204
Update 12/20: ok session-012_1751881258140 - 15.6ms - 204
Update 13/20: ok session-013_1751881258260 - 14.5ms - 204
Update 14/20: ok session-014_1751881258371 - 10.7ms - 204
Update 15/20: ok session-015_1751881258489 - 13.3ms - 204
Update 16/20: ok session-016_1751881258610 - 14.5ms - 204
Update 17/20: ok session

In [24]:
print("\nupdate proformance:")
analyze_performance_results(update_results)


update proformance:

OPERATION PERFORMANCE ANALYSIS
Total iterations:      20
Successful:            20 (100.0%)
Failed:                0 (0.0%)

Response Time Statistics (ms):
Minimum:               9.6 ms
Maximum:               24.9 ms
Average:               15.3 ms
Median (P50):          14.4 ms
95th Percentile:       24.9 ms
